
# <font color='blue'>***Fine Tuning do LLM Llama 2 Para Geração de Texto***</font>

## <font color='red'> ***OBSERVAÇÃO:***</font> ***Este Notebook deve ser executado em GPU.***

Pode ser Usado o Google Colab apenas com com o modelo base do Llama2.



###**Fine Tuning**

O  ajuste  fino  nos  permite  aproveitar  modelos  pré-treinados  existentes  e  adaptá-los  a tarefas  ou  domínios  específicos.  Ao  treinar  o  modelo  em  dados  específicos  de  uma  área  ou assunto, podemos adaptá-lo para um bom desempenho em tarefas específicas.

No  entanto,  este  processo  pode  consumir  muitos  recursos  e  ser  dispendioso,  pois modificaremos todos os  milhões de parâmetros, como parte do treinamento. O ajuste fino do modelo ainda requer muitos dados de treinamento, enorme infraestrutura e esforço.

E  no  processo  de  ajuste fino completo  deLLMs,  existe ainda o  risco  de  esquecimento catastrófico, onde se perde o conhecimento previamente adquirido no pré-treinamento.Ou seja, podemos ajustar um modelo por 8 horas para ao final descobrir que de fato ele “esqueceu” seu histórico de treinamento.

E  a aplicação  de  ajuste  fino  completo  a  um  único  modelo  para  diferentes  tarefas específicas de domínio geralmente resulta na criação de grandes modelos adaptados a tarefas específicas, sem modularidade.

 Logo, oque necessitamos é de uma abordagem modular que evite a alteração de todos os parâmetrosdo modelo, ao mesmo tempo que exige menos recursos de infraestrutura e menos dados.

 O  Parameter  Efficient  Fine  Tuning  (PEFT)  fornece  uma  maneira  de  realizar exatamente isso, o ajuste fino somente de parte dos parâmetros de um LLM, usando um volume de dados menor e também menor capacidade computacional. E, consequentemente, em menos tempo!

###**Parameter Efficient Fine Tuning (PEFT)**


Parameter Efficient Fine Tuning (PEFT)é uma técnica projetada para ajustar LLMs, e ao mesmo tempo, minimizar a necessidade de recursos e custos extensos.


PEFT é uma ótima opção ao lidar com tarefas específicas de domínio que necessitam de adaptação  do modelo.  Ao  empregar  o  PEFT,  podemos  encontrar  um  equilíbrio  entre  **reter conhecimento valioso do modelo pré-treinado e adaptá-lo** de forma eficaz à tarefa alvo com menos parâmetros.

###**Low-Rank Adaptation (LoRA)**

Low-Rank Adaptation (LoRA) um dos métodos do PEFT fornece uma abordagem modular para ajustar um LLM para tarefas específicas e fornece a capacidade de aprendizagem por transferência. A técnica LoRA pode ser implementada com menos recursos e é eficiente em termos de memória.

Aplicaremos um  adaptador  LoRA  para  as  redes  feed  forward  existentes no modelo.Vamos  congelar as  redes  feed  forward  originais  e  usaremos  a  rede  LoRA  para treinamento.

LoRA pode ser implementado como um adaptador projetado para aprimorar e expandir as camadas existentes de uma rede neural. O adaptador introduz uma camada adicional de parâmetros treináveis (pesos), mantendo os parâmetros originais em estado congelado. Esses parâmetros  treináveis  possuem  uma  classificação  (dimensão)  substancialmente  reduzida  em comparação  com  as  dimensões  da  rede  original.  Este  é  o  mecanismo  através  do  qual  LoRa simplifica e agiliza o processo de adaptação dos modelos originais para tarefas específicas de domínio.

O aspecto mais vantajoso desta abordagem é que o adaptador LoRA treinado pode ser preservado de forma independente e empregado como módulo distinto. Ao construir módulos específicos de domínio desta maneira, alcançamos efetivamente um alto nível de modularidade. Além disso, ao nos abstermos de alterar os pesos originais, contornamos com sucesso a questão do esquecimento catastrófico(quando o LLM “esquece”o que aprendeu no pré-treinamento).

Outras  melhorias podem  ser  implementadas  no  LoRA,  especialmente  por  meio  da utilização do QLoRA.

##**Quantized Low-Ranking Adaptation (QLoRA)**

QLoRA é uma extensão LoRA para aumentar a eficiência quantizando valores de peso da rede  original,  desde  tipos  de  dados  de  alta  resolução,  como  Float32,  até  tipos  de  dados  de resolução mais baixa, como int4. Isso leva a demandas reduzidas de memória e cálculos mais rápidos.

Existem três otimizações principais que o QLoRA traz além do LoRA, o que torna o QLoRA um dos melhores métodos PEFT: ***4-bit NF4 Quantization, Double Quantization e Unified Memory Paging***.Vejamos uma descrição de cada umdos métodos.


<font color='red'>**4-bit NF4 Quantization**<font>

A  quantização  NormalFloat4  de  4  bits  é  um  processo  de  3  etapas: **Normalização, Quantização e Desquantização.**

Como parte das etapas de normalização e quantização, os pesos são ajustados para uma média zero e uma variância unitária constante. Um tipo de dados de 4 bits pode armazenar apenas  16  números. Como  parte  da  normalização,  os  pesos  são  mapeados  para  esses  16 números, distribuídos com centro em zero e,em vez de armazenar os pesos, a posição mais próxima é armazenada.

Para Desquantizar os valores, fazemos exatamente o inverso.

Obviamente, há uma perda de dados quando normalizamos e quantizamos, à medida que passamos do FP32, que é um tipo de dados de alta resolução, para um tipo de dados de baixa resolução. A perda não é enorme, desde que não haja valores discrepantes no tensor de entrada, o  que  pode  afetar  o  absmax()  e  eventualmente  perturbar  a  distribuição.  Para  evitar  esse problema, geralmente quantizamos os pesos de forma independente por blocos menores, o que normalizará os valores discrepantes.

A quantização NormalFloat de 4 bits é aplicada aos pesos do modelo original, os pesos do adaptador LoRA serão FP32, pois todo o treinamento acontecerá nesses pesos. Feito todo o treinamento, os pesos originais serão desquantizados.


<font color='red'>**Double Quantization**<font>

A quantização dupla reduz ainda mais o consumo de memória, quantizando constantes de quantização. Na etapa anterior de quantização FP4 de 4 bits, calculamos a constante de quantização. Mesmo isso pode ser quantizado para melhor eficiênciae é isso que fazemos na Dupla Quantização.

Como a quantização é feita em blocos, para evitar outliers, normalmente 64 pesos em 1 bloco,  teremos  1  constante  de  quantização.  Essas  constantes  de  quantização  podem  ser quantizadas ainda mais, para reduzir o consumo de memória.


<font color='red'>**Unified Memory Paging**<font>

Juntamente  com  as  técnicas  acima,  o  QLoRA  também  utiliza  o  recurso  de  memória unificada da nVidia, que permite transferências contínuas de páginas GPU-> CPU, quando a GPU fica sem memória, gerenciando assim os picos repentinos de memória na GPU e ajudando em problemas de estouro/sobrecarga de memória.

LoRA e QLoRA são duas das técnicas mais amplamente utilizadas para ajuste fino com eficiência de parâmetros.


#**Instalação das Bibliotecas Necessárias**

In [ ]:
# Versão da Linguagem Python usada
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

Versão da Linguagem Python Usada Neste Jupyter Notebook: 3.10.12


In [ ]:
# Para atualizar um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install -U nome_pacote

# Para instalar a versão exata de um pacote, execute o comando abaixo no terminal ou prompt de comando:
# !pip install nome_pacote==versão_desejada

# Depois de instalar ou atualizar o pacote, reinicie o jupyter notebook.

# Instala o pacote watermark.
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.
!pip install -q -U watermark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 23.2 MB/s eta 0:00:00


O comando "!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118 --user" instala as bibliotecas PyTorch, torchvision e torchaudio no ambiente Python, sendo essenciais para aprendizado de máquina, visão computacional e processamento de áudio. A inclusão dos argumentos indica otimização para CUDA 11.8, relevante para GPUs NVIDIA, e instalação no escopo do usuário para evitar necessidade de permissões de administrador.

In [ ]:
#####. Tirar

#!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118 --user

**Instalações Abaixo:**

1. **accelerate (0.21.0):** Facilita o treinamento eficiente de modelos de aprendizado profundo, otimizando operações para acelerar o desempenho em GPUs e TPUs.
  
2. **peft (0.4.0):** Fornece ferramentas para avaliação de modelos de aprendizado de máquina, incluindo métricas e funções de desempenho.

3. **bitsandbytes (0.40.2):** Oferece utilitários para manipulação eficiente de bits e bytes, útil em diversas operações de codificação e processamento de dados.

4. **transformers (4.31.0):** Implementa modelos de linguagem pré-treinados e fornece interfaces para trabalhar com modelos de transformer, amplamente utilizados em processamento de linguagem natural.

5. **trl (0.4.7):** Introduz técnicas de aprendizado por reforço para ajuste fino de modelos de linguagem, permitindo a incorporação de recompensas durante o treinamento.

In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 62.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 53.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.5 MB/s eta 0:00:00


#**Importação das Bibliotecas**

1. **os:** Módulo para interação com o sistema operacional, utilizado para manipulação de caminhos e configurações.

2. **torch:** Biblioteca essencial para aprendizado de máquina e treinamento de modelos, fornecendo tensores e operações para GPU.

3. **peft:** Oferece ferramentas para avaliação de modelos de aprendizado de máquina, incluindo configurações personalizadas e modelos como o PeftModel.https://pypi.org/project/peft/

4. **trl:** Implementa técnicas de aprendizado por reforço para ajuste fino de modelos de linguagem, com o SFTTrainer sendo utilizado para treinamento.https://pypi.org/project/trl/

5. **datasets:** Facilita o carregamento de conjuntos de dados para treinamento e avaliação de modelos de linguagem.

6. **transformers:** Fornece implementações de modelos de linguagem, como AutoModelForCausalLM, e ferramentas úteis, como tokenizer e argumentos de treinamento, para trabalhar com modelos transformer baixado do Hugging Face.

7. **bitsandbytes:** Oferece configurações específicas, como BitsAndBytesConfig, para operações eficientes de manipulação de bits e bytes.https://pypi.org/project/bitsandbytes/

8. **warnings:** Controla a exibição de avisos durante a execução do código, filtrando avisos para melhorar a legibilidade do output.

   https://pypi.org/project/accelerate/.  

In [ ]:
# Imports
import pandas as pd
import os
import torch
import peft
import trl
import datasets
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
from datasets import load_dataset
from transformers import (AutoModelForCausalLM,
                          AutoTokenizer,
                          BitsAndBytesConfig,
                          HfArgumentParser,
                          TrainingArguments,
                          pipeline,
                          logging)
import warnings
warnings.filterwarnings('ignore')

**Versões dos pacotes usados neste jupyter notebook**

In [ ]:
%reload_ext watermark
%watermark -a "Radix / Celanese - GenAI" --iversions

Author: Radix / Celanese - GenAI

peft        : 0.4.0
torch       : 2.1.0+cu121
datasets    : 2.16.1
pandas      : 1.5.3
trl         : 0.4.7
transformers: 4.31.0



**Verificação de disponibilidade de GPU**

In [ ]:
if torch.cuda.is_available():
    print('Número de GPUs:', torch.cuda.device_count())
    print('Modelo GPU:', torch.cuda.get_device_name(0))
    print('Total Memória [GB] da GPU:',torch.cuda.get_device_properties(0).total_memory / 1e9)

Número de GPUs: 1
Modelo GPU: Tesla T4
Total Memória [GB] da GPU: 15.835660288


## **Definindo o Modelo Llama 2 Base**

**Llama-2-7b-chat-hf** será o modelo BASE utilizado neste primeiro passo que será baixado diretamente do hugging Face. Este modelo possui **7 Bilhões de parâmetros** é o menor disponível pelo Llama2, poderimos usar versões maiores, mas não temos GPU's, o modelo base do Llama2 é o que podemos rodar no Google Colab (gratuito) ele é **Open Source**, o que é bom para o cliente, pois não possui custos.

Há outros modelos que podem ser baixados no hugging Face, mas devemos baixar apenas modelos com documentação por motivos de segurança.

OBS: Esse modelo recebe apenas texto de entrada, diferente de outros LLM,s que são multi-Modal, deveremos analisar com diversos documentos e criar strings, mesmo para os valores numéricos (fase de testes).

____________

**Etapas para baixar o modelo:**


**1)** Criar conta huggingface.co;

**2)**Posteriormente aceitar os termos e condições da Meta na página https://huggingface.co/meta-llama/Llama-2-7b-chat-hf;

**3)**  Verificar no site toda a explicação do modelo que estaremos usando e como foi desenvolvido como material de referência.

<font color='red'>**Obs:**<font>  Havendo a liberação de GPU em nuvem será possível testar com outros modelos do Llama2. essa versão do Llama2 só aceita parquet

In [ ]:
##                                     >>>>>>>>>>> Pode ser Alterado <<<<<<<<<<<
# Nome do modelo Base
modelo_base = "NousResearch/Llama-2-7b-chat-hf"

# **Definindo o Dataset Para o Ajuste Fino**

- Para o ajuste fino, usaremos a nossa base de dados que foi preparada, ela possui hoje mais de 22 milhões de registros, mas inicialmente usaremos apenas 1000 registros aumentando gradativamente, devemos observar que o modelo só aceita base de dados em formato de texto, poderemos utilizar planilhas CSV desde que os dados estejam em formato de strings, ou arquivos TXT, PDF's, o objetivo é realizar diversos testes e preencher uma tabela de hipóteses os resultados.

_____________________

<font color='red'>**Atenção** uma estratégia adotada e colocar os dados em formato parquet, é um formato de compressão dos dados, acaba sendo muito melhor que usar o CSV, sendo uma tendência quando temos grandes conjuntos de dados.

<font color='red'>Podemos utilizar em formato CSV, mas irá impactar no desenvolvimento do ajuste fino do LLM.

<font color='red'>Poderemos gerar em CSV e posteriormente converter para parquet.

In [ ]:
##                                     >>>>>>>>>>> Pode ser Alterado <<<<<<<<<<<
# Dataset usado para o ajuste fino
# O dataset deve estar na nuvem do Hugging Face.

nome_dataset = 'JuniorBueno/txt'

## **Definindo Parâmetros LoRA**

* Como descrito no inicio, iremos utilizar a técnica Lora, sendo uma técnica de acelaração de ajuste fino de grandes modelos de LLM consumindo menos memória, então teremos que definir os três parâmetros:

  **1) Lora Attention;**

  **2) Lora Alpha;**  
  
  **3) Lora Dropout.**

______

* O Lora é otimizado para manter o numero de mudanças baixo, pois se tivermos que fazer muitas modificações de pesos no modelo, o treinamento levará muito tempo, estraté e manter baixo o número de mudanças.

_____

  **Link de referência para estudo:** https://huggingface.co/docs/peft/conceptual_guides/lora

* O parâmetro "Dimensão da Atenção LoRA" **(`lora_r`)** ajusta a extensão do foco de um modelo de linguagem, permitindo que ele dê mais importância a relações específicas em trechos próximos de texto. Isso ajuda a capturar nuances locais e a melhorar a compreensão de contextos específicos em linguagem natural.

In [ ]:
##                                     >>>>>>>>>>> Pode ser Alterado <<<<<<<<<<<
# Dimensão da atenção LoRA
lora_r = 64

* O parâmetro "Alpha para LoRA scaling" **(`lora_alpha`)** com valor 16 ajusta a importância de informações locais em um modelo de linguagem, permitindo destacar detalhes específicos e melhorar a capacidade do modelo de entender contextos próximos em texto. Este ajuste influencia como o modelo pondera a relevância das informações locais durante o processamento de dados.

In [ ]:
##                                     >>>>>>>>>>> Pode ser Alterado <<<<<<<<<<<
# Parâmetro Alpha para LoRA scaling
lora_alpha = 16

* O parâmetro "Probabilidade Dropout para as camadas LoRA" **(`lora_dropout`)** com valor 0.1 ajusta a quantidade controlada de informações que um modelo de linguagem retém durante o treinamento, ajudando a prevenir possíveis sobreajustes. Ao definir como 0.1, aproximadamente 10% das informações são aleatoriamente eliminadas, promovendo uma aprendizagem mais robusta e generalizável.

In [ ]:
##                                     >>>>>>>>>>> Pode ser Alterado <<<<<<<<<<<
# Probabilidade Dropout para as camadas LoRA
lora_dropout = 0.1

## **Definindo Parâmetros bitsandbytes**

* Serve para otimizar as operações matemáticas durante o ajute fino do LLM.


* O parâmetro "Ativa precisão 4-bits ao carregar o modelo base" **(`use_4bit = True`)** habilita o uso de uma representação mais compacta de números no modelo, reduzindo o consumo de memória e acelerando a inferência. Essa configuração otimiza o desempenho do modelo ao carregá-lo com uma precisão de 4 bits em vez da tradicional precisão de 32 bits.

In [ ]:
# Ativa precisão 4-bits ao carregar o modelo base
use_4bit = True

* O parâmetro "Define o dtype para os parâmetros do modelo base" **(`bnb_4bit_compute_dtype = "float16"`)** ajusta a representação numérica utilizada nos cálculos internos do modelo para uma versão mais compacta, otimizando o uso de memória e acelerando o processamento. Isso resulta em uma execução mais eficiente do modelo ao utilizar precisão de 16 bits em vez da tradicional precisão de 32 bits.

In [ ]:
# Define o dtype para os parâmetros do modelo base
bnb_4bit_compute_dtype = "float16"

Quantização: É um processo de redução da precisão dos números usados em um modelo de Machine Learning. Por exemplo, em vez de usar números de ponto flutuante de 32 bits (como FP32), a quantização pode reduzir esses números para representações de 16 bits, 8 bits, ou até menos. O objetivo é diminuir o tamanho do modelo e aumentar a velocidade de inferência, mantendo ao máximo a precisão do modelo.

**Vantagens:** https://arxiv.org/abs/2208.07339

In [ ]:
# Tipo de quantização (fp4 ou nf4)
# Podemos usar o tipo NF4, que é um tipo de dados de 4 bits adaptado para pesos que foram inicializados usando uma distribuição normal.
# NF4 = Normal Float 4
bnb_4bit_quant_type = "nf4"

In [ ]:
# Desativa a quantização aninhada para modelos básicos de 4 bits (quantização dupla)
use_nested_quant = False

## Definindo Parâmetros de Treinamento

* Não há nada compleyamente difinido, devemos fazer experimentação com diverssos parâmetros.

**OBS:** Nunca faça alteração em diversos parâmetros e de uma só vez, pois desta forma será muito dificil identificar qual alteração realizada foi eficaz ou não para o modelo.

* Criar o diretório de saída

In [ ]:
# Diretório de saída onde as previsões e os checkpoints do modelo serão armazenados
output_dir = "resultados"

* O parâmetro "Número de épocas de treino" **(`num_train_epochs = 2`)**
 define por quantos ciclos completos de treinamento o modelo passará pelos dados. Com um valor de 2, o modelo será treinado duas vezes com o conjunto de dados completo, ajustando seus parâmetros para aprender padrões e melhorar o desempenho ao longo dessas iterações.

In [ ]:
##                                   >>>>>>>>>>> Pode ser Alterado <<<<<<<<<<<
# Número de épocas de treino
num_train_epochs = 10

* Os parâmetros "treinamento **fp16/bf16"** definem a precisão numérica utilizada durante o treinamento de modelos de aprendizado de máquina. Se "bf16" estiver configurado como True com um hardware específico, como o A100, a representação de ponto flutuante com 16 bits (bf16) é utilizada para otimizar o desempenho do treinamento, enquanto "fp16" configurado como False indica que a representação de 16 bits não está ativa.



https://en.wikipedia.org/wiki/Bfloat16_floating-point_format

In [ ]:
# Habilita o treinamento fp16/bf16 (defina bf16 como True com um A100)
fp16 = False
bf16 = True

* O parâmetro "Batch size por GPU para treino" **(`per_device_train_batch_size = 4`)** define quantos exemplos de dados são processados simultaneamente por cada unidade de processamento gráfico (GPU) durante o treinamento de um modelo. Um valor de 4 significa que cada GPU processa 4 exemplos ao mesmo tempo, influenciando a eficiência e velocidade do treinamento do modelo.

In [ ]:
##                                     >>>>>>>>>>> Pode ser Alterado <<<<<<<<<<<
# Batch size por GPU para treino
per_device_train_batch_size = 4

* O parâmetro "Batch size por GPU para avaliação" **(`per_device_eval_batch_size = 4`)** define quantos exemplos de dados são processados simultaneamente por cada unidade de processamento gráfico (GPU) durante a fase de avaliação de um modelo. Um valor de 4 significa que cada GPU processa 4 exemplos ao mesmo tempo, impactando a eficiência e velocidade da avaliação do modelo.

In [ ]:
##                                     >>>>>>>>>>> Pode ser Alterado <<<<<<<<<<<
# Batch size por GPU para avaliação
per_device_eval_batch_size = 4

* O parâmetro "Número de etapas de atualização para acumular os gradientes" **(`gradient_accumulation_steps = 1`)** controla com que frequência os gradientes (informações sobre como ajustar o modelo) são atualizados durante o treinamento. Um valor de 1 significa que os gradientes são atualizados a cada etapa, influenciando a eficácia do treinamento e o uso de recursos computacionais.

In [ ]:
##                                     >>>>>>>>>>> Pode ser Alterado <<<<<<<<<<<
# Número de etapas de atualização para acumular os gradientes
gradient_accumulation_steps = 1

* O parâmetro "Habilita gradient checkpointing" **(`gradient_checkpointing = True`)** ativa uma técnica que gerencia eficientemente a memória durante o treinamento de modelos, especialmente útil para modelos grandes. Isso pode melhorar a eficiência do treinamento ao reduzir os requisitos de memória, possibilitando o treinamento de modelos maiores com menos restrições computacionais.

In [ ]:
# Habilita gradient checkpointing
gradient_checkpointing = True

* O parâmetro "Maximum gradient normal (gradient clipping)" **(`max_grad_norm = 0.3`)** define um limite para a magnitude dos gradientes durante o treinamento de um modelo. Essa limitação ajuda a evitar problemas como explosão de gradientes, contribuindo para um treinamento mais estável e prevenindo valores extremos que podem afetar a eficácia do modelo.

In [ ]:
##                                     >>>>>>>>>>> Pode ser Alterado <<<<<<<<<<<
# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

* O parâmetro "Taxa de aprendizado" **(`learning_rate = 2e-4`)** determina o tamanho dos passos que um modelo de aprendizado de máquina dá ao ajustar seus parâmetros durante o treinamento. Um valor como 2e-4 indica uma taxa de aprendizado relativamente pequena, controlando a velocidade e estabilidade do processo de aprendizado do modelo.

In [ ]:
# Taxa de aprendizado
learning_rate = 2e-4

* O parâmetro "Redução de peso" **(`weight_decay = 0.001`)** é uma técnica durante o treinamento de modelos de aprendizado de máquina para reduzir o impacto de pesos desnecessários, promovendo uma generalização melhor. O valor 0.001 indica a intensidade da redução, sendo uma abordagem para evitar sobreajuste e melhorar o desempenho do modelo em dados novos.

In [ ]:
# Redução de peso a ser aplicada a todas as camadas, exceto pesos bias/LayerNorm
weight_decay = 0.001

* O parâmetro "Otimizador" **(`optim = "paged_adamw_32bit"`)** define o algoritmo utilizado para ajustar um modelo durante o treinamento. "paged_adamw_32bit" especifica um otimizador específico com configurações específicas, influenciando como o modelo aprende a partir dos dados. A escolha do otimizador pode afetar o desempenho e a eficiência do treinamento do modelo.

In [ ]:
# Otimizador
optim = "paged_adamw_32bit"

Ao usar **lr_scheduler_type = "cosine"**, está-se especificando que o agendador de taxa de aprendizagem deve seguir esta estratégia de cosseno para ajustar a taxa de aprendizagem durante o treinamento do modelo. Essa abordagem é muitas vezes eficaz para obter uma convergência mais rápida e estável do modelo.

In [ ]:
# Learning rate schedule
lr_scheduler_type = "cosine"

* O parâmetro "Número de etapas de treinamento" **(`max_steps = -1`)** determina por quantas iterações o modelo será treinado. Quando definido como -1, indica que o treinamento deve continuar até que o critério de parada adequado seja atendido, sem um número fixo predefinido de etapas. Essa flexibilidade permite adaptar o treinamento conforme necessário.

In [ ]:
# Número de etapas de treinamento
max_steps = -1

O warmup_ratio especifica a proporção do treinamento total que será dedicada ao aquecimento. Por exemplo, um **warmup_ratio de 0.03** significa que 3% do treinamento total será usado para aquecer a taxa de aprendizagem. Se o treinamento total for de 100 épocas, por exemplo, o aquecimento ocorrerá nas primeiras 3 épocas.

Durante a fase de aquecimento, a taxa de aprendizagem aumenta linearmente de um valor inicial (geralmente próximo de zero) até o valor de taxa de aprendizagem estabelecido para o treinamento regular. Após o período de aquecimento, a taxa de aprendizagem segue o plano estabelecido, que pode ser constante, decrescente ou seguindo algum outro agendador.

In [ ]:
# Proporção de passos para um aquecimento linear (de 0 à taxa de aprendizagem)
warmup_ratio = 0.03

* O parâmetro "Agrupar sequências em lotes com o mesmo comprimento" **(`group_by_length = True`)** organiza as sequências de dados em lotes, garantindo que sequências de comprimentos similares sejam agrupadas juntas. Isso otimiza o uso de memória e acelera o treinamento, facilitando o processamento eficiente de diferentes comprimentos de sequências durante o treinamento do modelo.

In [ ]:
# Agrupar sequências em lotes com o mesmo comprimento
# Economiza memória e acelera consideravelmente o treinamento
group_by_length = True

* O parâmetro "Salvar checkpoint a cada X etapas de atualização" **(`save_steps = 0`)** determina com que frequência o progresso do treinamento é salvo em um ponto específico, chamado checkpoint. Um valor de 0 significa que nenhum checkpoint intermediário será salvo durante o treinamento. A utilização de checkpoints é uma prática para proteger o progresso do treinamento, permitindo a retomada a partir de pontos específicos em caso de interrupções ou falhas.

In [ ]:
# Salvar checkpoint a cada X etapas de atualização
save_steps = 0

* O parâmetro "Registrar cada X etapas de atualizações" **(`logging_steps = 25`)** controla com que frequência informações sobre o treinamento, como métricas de desempenho, são registradas e exibidas. Com um valor de 25, as atualizações do treinamento são registradas a cada 25 passos, oferecendo uma visão resumida e gerenciável do progresso do modelo durante o treinamento.

In [ ]:
# Registrar cada X etapas de atualizações
logging_steps = 25

# **Definindo Parâmetros do Ajuste Fino**

https://huggingface.co/docs/trl/sft_trainer

O parâmetro "Comprimento máximo de sequência a ser usado" **(`max_seq_length = None`)** determina o tamanho máximo que uma sequência de dados pode ter durante o treinamento de um modelo. Ao definir como "None", não há restrição de comprimento, permitindo que o modelo processe sequências de qualquer tamanho, o que pode ser útil para lidar com dados variáveis em comprimento.

In [ ]:
# Comprimento máximo de sequência a ser usado
max_seq_length = None

* O parâmetro "Reunir vários exemplos curtos na mesma sequência de entrada para aumentar a eficiência" **(`packing = False`)** determina se várias instâncias curtas de dados serão agrupadas em uma única entrada durante o treinamento do modelo. Quando configurado como "False", cada exemplo curto é tratado individualmente, enquanto "True" indicaria agrupamento para otimizar o processo de treinamento.

In [ ]:
# Reunir vários exemplos curtos na mesma sequência de entrada para aumentar a eficiência
packing = False

* O parâmetro "Carregar o modelo inteiro na GPU 0" (`device_map = {"": 0}`) especifica que o modelo e seus dados devem ser processados exclusivamente na GPU 0. Essa configuração é relevante para usuários com múltiplas unidades de processamento gráfico (GPUs), permitindo direcionar o processamento para uma GPU específica.

In [ ]:
# Carregar o modelo inteiro na GPU 0
device_map = {"": 0}

* O parâmetro "Carrega o dataset com a amostra de treino" **(`dataset = load_dataset(nome_dataset, split="train")`)** carrega os dados de treinamento de um conjunto de dados chamado `nome_dataset`. Isso é útil para preparar os dados necessários para treinar um modelo de aprendizado de máquina, fornecendo a amostra de treino específica do conjunto de dados.

In [ ]:
# Carrega o dataset com a amostra de treino
dataset = load_dataset(nome_dataset, split = "train")

Generating train split: 0 examples [00:00, ? examples/s]

* O parâmetro carrega o tokenizer e modelo com configuração QLoRA **(`compute_dtype = getattr(torch, bnb_4bit_compute_dtype)`)** configura o tipo de dados utilizado durante a computação no modelo, seguindo a especificação QLoRA. Isso influencia a representação numérica utilizada internamente, podendo otimizar o modelo para eficiência computacional.

In [ ]:
# Carrega o tokenizer e modelo com configuração QLoRA
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

* O parâmetro colocamos todos os parâmetros que já definimos anteriormente em **"bnb_config"** é uma configuração para um modelo que utiliza BitsAndBytes (BNB), permitindo a personalização de características como o uso de representação em 4 bits, tipo de quantização e precisão computacional. Essas configurações influenciam como o modelo processa dados, otimizando-o para eficiência e desempenho.


### **Os parâmetros:**


- **`load_in_4bit`** controla se os dados são carregados no modelo usando representação de 4 bits;
- **`bnb_4bit_quant_type`** especifica o tipo de quantização de 4 bits;
- **`bnb_4bit_compute_dtype`** define a precisão numérica interna para cálculos;
- **`bnb_4bit_use_double_quant`** ativa a técnica de quantização dupla para otimizar a eficiência do modelo, sendo essenciais para configurar a eficácia e desempenho do modelo de forma personalizada.

In [ ]:
bnb_config = BitsAndBytesConfig(load_in_4bit = use_4bit,
                                bnb_4bit_quant_type = bnb_4bit_quant_type,
                                bnb_4bit_compute_dtype = compute_dtype,
                                bnb_4bit_use_double_quant = use_nested_quant)

* O trecho de código verifica se a GPU é compatível com a representação bfloat16 (bf16), uma técnica de representação numérica. Se a GPU suportar bfloat16, uma mensagem é exibida indicando que é possível acelerar o treinamento do modelo ativando a opção `bf16=True`, o que pode melhorar a eficiência do processo de treinamento em determinadas circunstâncias.

In [ ]:
# Verificando se a GPU é compatível com bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("A GPU suporta bfloat16: acelere o processo de treino com bf16=True")
        print("=" * 80)

# **Carregando o Modelo Base e o Tokenizador**

* O parâmetro **`modelo_celanese`** carrega um modelo de linguagem pré-treinado para uso em tarefas específicas, como geração de texto. **`quantization_config`** e **`device_map`** configuram detalhes internos do modelo, como a representação numérica e o dispositivo de processamento, afetando sua eficiência e desempenho. Essas configurações são importantes ao utilizar o modelo em diferentes contextos.

In [ ]:
# Carrega o modelo base
modelo_celanese = AutoModelForCausalLM.from_pretrained(modelo_base,
                                                  quantization_config = bnb_config,
                                                  device_map = device_map)

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/179 [00:00<?, ?B/s]

* O parâmetro **`modelo_celanese.config.use_cache`** desativa o armazenamento em cache do modelo, o que significa que ele não salvará resultados intermediários para uso posterior. Isso pode acelerar a execução do modelo, mas resultará na reexecução de tarefas sempre que o modelo for utilizado. A desativação do cache pode ser útil em situações onde a memória é uma preocupação e os resultados intermediários não precisam ser retidos.

In [ ]:
modelo_celanese.config.use_cache = False

* O parâmetro **`modelo_dsa.config.pretraining_tp`** ajusta um aspecto específico do treinamento prévio do modelo, mas sem detalhes adicionais, é difícil fornecer uma descrição específica. Pode estar relacionado a configurações específicas ou hiperparâmetros do treinamento prévio, influenciando o comportamento geral do modelo nessa fase de aprendizado. O valor 1 atribuído a esse parâmetro indica uma configuração específica de pré-treinamento.

In [ ]:
modelo_celanese.config.pretraining_tp = 1

* Um **tokenizador** em Processamento de Linguagem Natural (PLN) é uma ferramenta que divide o texto em unidades significativas chamadas tokens, como palavras ou subpalavras, facilitando a análise e processamento de linguagem por modelos de machine learning.

* Neste caso esse modelo foi treinado com um tokenizador para desenvolver o modelo, então será necessário baixar o mesmo modelo no qual foi desenvolvido para o nosso desenvolvimento.

* O parâmetro **`tokenizer`** prepara um tokenizador automático para processar texto usando um modelo pré-treinado chamado **`modelo_celanese`**. A configuração adicional, como **`tokenizer.pad_token`** e **`tokenizer.padding_side`**, ajusta como o tokenizador lida com o preenchimento de sequências de texto durante o processamento.

In [ ]:
# Carrega o tokenizador do LLaMA2
tokenizer = AutoTokenizer.from_pretrained(modelo_base, trust_remote_code = True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

# **Carregando a Configuração LoRA e SFTTrainer**

* O parâmetro **`peft_config`** configura um modelo com atenção localizada (LoRA) para tarefas de linguagem, ajustando aspectos como a intensidade da atenção **(`lora_alpha`)**, a probabilidade de exclusão de informações **(`lora_dropout`)**, e outros detalhes específicos do modelo. Esse tipo de configuração é fundamental para personalizar o comportamento do modelo em tarefas específicas de processamento de linguagem natural.

In [ ]:
# Carrega a configuração LoRA
peft_config = LoraConfig(lora_alpha = lora_alpha,
                         lora_dropout = lora_dropout,
                         r = lora_r,
                         bias = "none",
                         task_type = "CAUSAL_LM")

* O parâmetro **`training_arguments`** configura diversos aspectos do treinamento de um modelo, como a duração do treinamento, tamanho dos lotes de dados, otimização, entre outros. Essas configurações são essenciais para controlar o processo de aprendizado do modelo, adaptando-o às necessidades específicas da tarefa e otimizando sua eficiência.

* **`output_dir`**: Define o diretório onde os resultados do treinamento, como modelos e métricas, serão salvos.
- **`num_train_epochs`**: Especifica por quantas épocas o modelo será treinado, determinando a extensão do processo de aprendizado.
- **`per_device_train_batch_size`**: Define o tamanho dos lotes de dados processados por vez durante o treinamento, influenciando a eficiência do modelo.
- **`gradient_accumulation_steps`**: Controla com que frequência os ajustes nos parâmetros do modelo são atualizados, afetando a estabilidade do treinamento.
- **`optim`**: Especifica o otimizador utilizado para ajustar o modelo durante o treinamento, influenciando a eficácia da aprendizagem.
- **`save_steps`** e **`logging_steps`**: Determinam a frequência de salvamento de resultados intermediários e de registros, respectivamente, durante o treinamento.
- **`learning_rate`** e **`weight_decay`**: Ajustam a taxa de aprendizado e a redução de peso, influenciando a velocidade e estabilidade do treinamento.
- **`fp16`** e **`bf16`**: Ativam ou desativam a representação numérica em 16 bits para otimizar a eficiência do treinamento.
- **`max_grad_norm`**: Define um limite para a magnitude dos ajustes nos parâmetros, evitando problemas de estabilidade durante o treinamento.
- **`max_steps`**: Especifica o número máximo de passos de treinamento, controlando a duração total do processo.
- **`warmup_ratio`**: Define a proporção de etapas iniciais durante as quais a taxa de aprendizado é aumentada gradualmente.
- **`group_by_length`**: Agrupa sequências de comprimentos semelhantes para otimizar o treinamento em termos de eficiência.
- **`lr_scheduler_type`**: Especifica o tipo de programador de taxa de aprendizado utilizado durante o treinamento.
- **`report_to`**: Indica onde os resultados do treinamento serão registrados, como em um painel TensorBoard.

In [ ]:
# Parâmetros de configuração de treino
training_arguments = TrainingArguments(output_dir = output_dir,
                                       num_train_epochs = num_train_epochs,
                                       per_device_train_batch_size = per_device_train_batch_size,
                                       gradient_accumulation_steps = gradient_accumulation_steps,
                                       optim = optim,
                                       save_steps = save_steps,
                                       logging_steps = logging_steps,
                                       learning_rate = learning_rate,
                                       weight_decay = weight_decay,
                                       fp16 = True,
                                       bf16 = False,
                                       max_grad_norm = max_grad_norm,
                                       max_steps = max_steps,
                                       warmup_ratio = warmup_ratio,
                                       group_by_length = group_by_length,
                                       lr_scheduler_type = lr_scheduler_type,
                                       report_to = "tensorboard")

- **`model`**: Especifica o modelo de linguagem a ser treinado, usando um modelo pré-treinado chamado `modelo_celanese`.
- **`train_dataset`**: Define o conjunto de dados de treinamento, que contém exemplos de texto usados para ensinar o modelo.
- **`peft_config`**: Configura o modelo com parâmetros específicos para atenção localizada, influenciando seu comportamento durante o treinamento.
- **`dataset_text_field`**: Indica o campo do conjunto de dados que contém o texto a ser processado pelo modelo.
- **`max_seq_length`**: Limita o comprimento máximo das sequências de texto durante o treinamento, otimizando o processamento.
- **`tokenizer`**: Prepara um tokenizador automático para dividir o texto em unidades significativas.
- **`args`**: Inclui as configurações detalhadas do treinamento, como duração, tamanho dos lotes e otimização.
- **`packing`**: Determina se várias instâncias curtas de dados são agrupadas para eficiência durante o treinamento.

In [ ]:
# Definir parâmetros de ajuste fino com SFTTrainer
trainer = SFTTrainer(model = modelo_celanese,
                     train_dataset=dataset,
                     peft_config=peft_config,
                     dataset_text_field="Timestamp",
                     max_seq_length=max_seq_length,
                     tokenizer=tokenizer,
                     args = training_arguments,
                     packing = packing)

Map:   0%|          | 0/22680000 [00:00<?, ? examples/s]

# **Treinamento e Ajuste Fino**

"**Training loss" (perda de treinamento)** é uma métrica usada durante o treinamento de um modelo de machine learning. Ela representa a medida de quão bem o modelo está performando em relação à tarefa específica para a qual está sendo treinado. A perda é uma forma de quantificar o quão longe as predições do modelo estão dos rótulos ou valores reais do conjunto de treinamento.

Durante o treinamento de um modelo, a perda é calculada em cada iteração (ou lote) com base nas previsões feitas pelo modelo em comparação com os rótulos conhecidos. O objetivo do treinamento é ajustar os parâmetros do modelo de modo a minimizar essa perda. Em outras palavras, o modelo está sendo ajustado para fazer previsões que se aproximem o máximo possível dos valores reais.

A perda pode ser calculada de diversas maneiras, dependendo da tarefa em questão. Por exemplo, em problemas de classificação, a perda pode ser medida usando a entropia cruzada, enquanto em problemas de regressão, a perda pode ser a média do erro quadrático.

Ao longo do treinamento, os valores da perda podem ser monitorados para avaliar o progresso do modelo. Um decréscimo consistente na perda geralmente indica que o modelo está aprendendo e se ajustando bem aos dados de treinamento. No entanto, é importante também avaliar o desempenho do modelo em conjuntos de validação ou teste para garantir que ele generalize bem para dados não vistos.

In [ ]:
# Treina o modelo
trainer.train()

Step,Training Loss
25,1.610100
50,1.602700
75,1.597800
100,1.621300
125,1.623600
150,1.621400
175,1.620500
200,1.637300
225,1.610500
250,1.626400


In [ ]:
# Salva o modelo ajustado
trainer.model.save_pretrained("llama-2-7b-ajustado")

## Gerando Texto com o Modelo Ajustado

In [ ]:
# Ignora warnings
logging.set_verbosity(logging.CRITICAL)

**Prompt = Pergunta**

In [ ]:
prompt = "How was the Process Dew Point last month?"

- Criaçao da execuçao de pipeline com o modelo criado Celanese que é o nosso LLM ajustado;
- Preparamos o tokenizador = tokenizer;



In [ ]:
# Cria o pipeline
pipe = pipeline(task = "text-generation",
                model = modelo_celanese,
                tokenizer = tokenizer,
                max_length = 200)

**Execução do pipeline que foi criado com o prompt**

In [ ]:
# Executa o pipeline e gera o texto a partir do prompt inicial
resultado = pipe(f"<s>[INST] {prompt} [/INST]")

##**Resposta:**

In [ ]:
print(resultado[0]['generated_text'])

# Fim